In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace

In [2]:
spark = SparkSession.builder\
	.master("local[*]").appName("IDS_Analysis").getOrCreate()

24/08/30 22:00:38 WARN Utils: Your hostname, abraham resolves to a loopback address: 127.0.1.1; using 192.168.0.145 instead (on interface eno2)
24/08/30 22:00:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/30 22:00:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/30 22:00:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
file_path = "/user/ids_analysis/UNSW-NB15.csv"
data = spark.read.csv(file_path, inferSchema=True)
data = data.toDF(
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbytes',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans_depth',
    'res_bdy_len',
    'Sjit',
    'Djit',
    'Stime',
    'Ltime',
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ ltm',
    'ct_src_dport_ltm',
    'ct_dst_sport_ltm',
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'
)

In [4]:
## data description
data.describe().show()

24/08/30 22:01:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------+-----------+------------------+-----------+------------------+-------+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-------+--------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-----------------+-----------------+-----------------+-----------------+-------

In [9]:
attack_cats = data.select('attack_cat').distinct().collect()

In [14]:
attack_cats

[Row(attack_cat='Worms'),
 Row(attack_cat=None),
 Row(attack_cat='Analysis'),
 Row(attack_cat='Shellcode '),
 Row(attack_cat='DoS'),
 Row(attack_cat='Reconnaissance '),
 Row(attack_cat='Backdoor'),
 Row(attack_cat='Fuzzers '),
 Row(attack_cat='Exploits'),
 Row(attack_cat='Generic'),
 Row(attack_cat='Shellcode'),
 Row(attack_cat='Backdoors'),
 Row(attack_cat='Fuzzers'),
 Row(attack_cat='Reconnaissance')]

In [26]:
# rename duplicate attack categories
for i in range(len(attack_cats)-1):
    if " " in str(attack_cats[i][0]):
        print(attack_cats[i][0])


Shellcode 
Reconnaissance 
Fuzzers 


In [24]:
range(len(attack_cats)-1)

range(0, 13)

In [27]:
clean_data = data.withColumn("attack_cat", regexp_replace("attack_cat", "Shellcode ", "Shellcode"))
clean_data = clean_data.withColumn("attack_cat", regexp_replace("attack_cat", "Reconnaissance ", "Reconnaissance"))
clean_data = clean_data.withColumn("attack_cat", regexp_replace("attack_cat", "Fuzzers ", "Fuzzers"))


In [28]:
clean_attack_cats = clean_data.select('attack_cat').distinct().collect()

In [29]:
len(clean_attack_cats)

11

In [30]:
clean_attack_cats

[Row(attack_cat='Worms'),
 Row(attack_cat='Shellcode'),
 Row(attack_cat=None),
 Row(attack_cat='Fuzzers'),
 Row(attack_cat='Analysis'),
 Row(attack_cat='DoS'),
 Row(attack_cat='Reconnaissance'),
 Row(attack_cat='Backdoor'),
 Row(attack_cat='Exploits'),
 Row(attack_cat='Generic'),
 Row(attack_cat='Backdoors')]

In [32]:
# save cleaned data
clean_data.write.csv("/user/ids_analysis/cleaned_data.csv")